# 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

from importlib import reload
import utils.utils as utils
reload(utils)  # Recarga el módulo

<module 'utils.utils' from 'c:\\Users\\USUARIO\\OneDrive - Universidad de Antioquia\\Aprendizaje\\Universidad\\2023-2\\Analítica II\\Caso Estudio No Supervisado\\no_supervisado\\utils\\utils.py'>

# 2. Importar base de datos

In [2]:
df2 = pd.read_csv('datasets/data_limpio.csv')

In [3]:
df2.head()

,tipo_entidad,fecha_corte,unicap,desc_unicap,renglon,desc_renglon,saldo_a_la_fecha,vigente,clientes_mora+30d,riesgo_a_c,...,riesgo_b_s,riesgo_c_c,riesgo_c_s,riesgo_d_c,riesgo_d_s,riesgo_e_c,riesgo_e_s,venc1_6,venc6_12,venc+12
0,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,37651351298.00,34981440061.00,469,6176,...,1835206170.00,142,975343138.00,326,1368695340.00,69,593093538.00,2425836044.00,244075193.00,0.00
1,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,826852228377.00,782016598064.00,5830,125128,...,11740703689.00,1486,9872854505.00,1854,14111818486.00,2469,18460447300.00,34361461982.00,10474168331.00,0.00
2,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,108451564795.00,104345665139.00,425,9723,...,2920771751.00,207,1936819277.00,165,1623502258.00,144,1544005629.00,3441821351.00,664078305.00,0.00
3,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,209307777744.00,196907738953.00,9361,91873,...,6541005114.00,2385,2574770601.00,2515,3677977262.00,160,382597983.00,12233133337.00,166905454.00,0.00
4,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,243818014709.00,227311255658.00,2295,32098,...,5969178631.00,848,5034273159.00,521,3607315495.00,520,4302662628.00,12919067892.00,3587691159.00,0.00


In [4]:
df2['unicap'] = df2['unicap'].astype('object')
df2['renglon'] = df2['renglon'].astype('object')
df2['fecha_corte'] = pd.to_datetime(df2['fecha_corte'])

### Tomar una muestra de tamaño 5000

In [5]:
utils.univariado_barras(df2, 'tipo_entidad', orientation='h')

,tipo_entidad,Frecuencia Absoluta,Frecuencia Relativa
3,ESTABLECIMIENTOS BANCARIOS,56116,69.365 %
0,COMPAÑIAS DE FINANCIAMIENTO,13900,17.182 %
2,ENTIDADES COOPERATIVAS DE CARÁCTER FINANCIERO,8614,10.648 %
4,INSTITUCIONES OFICIALES ESPECIALES,2153,2.661 %
1,CORPORACIONES FINANCIERAS,117,0.145 %


In [6]:
df = df2.sample(25000, random_state=123)

In [7]:
utils.univariado_barras(df, 'tipo_entidad', orientation='h')

,tipo_entidad,Frecuencia Absoluta,Frecuencia Relativa
3,ESTABLECIMIENTOS BANCARIOS,17305,69.22 %
0,COMPAÑIAS DE FINANCIAMIENTO,4278,17.112 %
2,ENTIDADES COOPERATIVAS DE CARÁCTER FINANCIERO,2733,10.932 %
4,INSTITUCIONES OFICIALES ESPECIALES,647,2.588 %
1,CORPORACIONES FINANCIERAS,37,0.148 %


# 3. Correlaciones variables numéricas

In [8]:
cols_num = df.select_dtypes(exclude=['object', 'datetime'])
df_corr = cols_num.corr()
fig = px.imshow(df_corr[(df_corr > 0.7) | (df_corr < -0.7)],  text_auto=True)
fig.update_layout(height=800)
fig.show()

## ¿Por qué el saldo a la fecha y el saldo vigente presentan una correlación tan alta?

El saldo a la fecha y el saldo vigente presentan una alta correlación. El saldo vigente representa el saldo luego de los abonos de los clientes a cada una de las cuentas en cada mes. Por tanto, se decide descartar el saldo vigente ya que saldo a la fecha y los abonos nos brindan la misma información.

In [9]:
df.drop('vigente', axis=1, inplace=True)

## ¿Qué hacer con las correlaciones altas en algunos riesgos?

In [10]:
df['year_month'] = pd.to_datetime(df.fecha_corte.apply(lambda x: str(x.year) + '-' + str(x.month) + '-' + '01'))
base = df.groupby(['year_month'])[['saldo_a_la_fecha','riesgo_a_s','riesgo_b_s','riesgo_c_s','riesgo_d_s','riesgo_e_s']].mean()
fig = px.line(base)
fig.show()

In [11]:
base2 = df.groupby(['year_month'])[['riesgo_a_c','riesgo_b_c','riesgo_c_c','riesgo_d_c','riesgo_e_c']].mean()
fig = px.line(base2)
fig.show()

La gran mayoría de los clientes tienen obligaciones catalogadas como de riesgo a, que es la categoría de menos riesgo para las entidades. Dado que las columnas saldo por riesgo representan cada una el saldo total de cuentas que hay de dichos riesgos y la gran mayoría son de riesgos a, hay una correlación casi de 100% entre el saldo de cuentas de riesgo a y el saldo total a la fecha, se puede observar que varían casi que igual en el tiempo. Dicho lo anterior, se decide conservar el saldo total a la fecha y descartar las columnas de saldo por riesgos. Las columnas de número de clientes por riesgos también representa muy bien la frecuencia e importancia de cada riesgos.

In [12]:
df.drop(['riesgo_a_s', 'riesgo_b_s', 'riesgo_c_s', 'riesgo_d_s', 'riesgo_e_s'], axis=1, inplace=True)

## ¿Qué hacer con las correlaciones altas de los riesgos con el número de clientes con más de 1 mes en mora?

In [13]:
base3 = df.groupby(['year_month'])[['clientes_mora+30d','riesgo_a_c','riesgo_b_c','riesgo_c_c','riesgo_d_c','riesgo_e_c']].mean()
fig = px.scatter_matrix(base3)
fig.update_layout(height=800)
fig.show()

Dado que esta variable presenta una alta correlación con otras 4 variables, se decide descartarla.

In [14]:
df.drop(['clientes_mora+30d'], axis=1, inplace=True)

## Observar correlación

In [15]:
cols_num = df.select_dtypes(exclude=['object', 'datetime'])
df_corr = cols_num.corr()
fig = px.imshow(df_corr,  text_auto=True)
fig.update_layout(height=1000)
fig.show()

# 4. Variables categóricas

In [16]:
df_cat = df.select_dtypes(include='object')
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 19012 to 42578
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tipo_entidad  25000 non-null  object
 1   unicap        25000 non-null  object
 2   desc_unicap   25000 non-null  object
 3   renglon       25000 non-null  object
 4   desc_renglon  25000 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [17]:
from scipy.stats import chi2_contingency
tabla_contingencia1 = pd.crosstab(df['tipo_entidad'], df['desc_unicap'])
tabla_contingencia2 = pd.crosstab(df['tipo_entidad'], df['desc_renglon'])
tabla_contingencia3 = pd.crosstab(df['desc_renglon'], df['desc_unicap'])
chi21, p1, _, _ = chi2_contingency(tabla_contingencia1)
chi22, p2, _, _ = chi2_contingency(tabla_contingencia2)
chi23, p3, _, _ = chi2_contingency(tabla_contingencia3)

print('tipo_entidad vs desc_unicap:', p1)
print('tipo_entidad vs desc_renglon:', p2)
print('desc_renglon vs desc_unicap:', p3)

tipo_entidad vs desc_unicap: 0.0
tipo_entidad vs desc_renglon: 0.0
desc_renglon vs desc_unicap: 0.0


Las tres variables presentan correlación, ya que es p-valor de las pruebas chi cuadrado fue menor a 0.05.

El tipo de entidad son los clusters a los que queremos llegar con el análisis, por tanto, esta no se tiene en cuenta para descartarla por su correlación. Decidimos eliminar unicap, ya que renglon son categorías dentro de cada categoría de unicap, por tanto, contiene información más detallada sobre los tipos de productos.

In [18]:
df.drop(['unicap','desc_unicap'], axis=1, inplace=True)

# 5. Exportar data sin correlación

In [19]:
df.to_csv('datasets/data_no_corr.csv', index=False)